# Complexity vs MSE and Rank Graphs


In [1]:
%pip install safetensors

Note: you may need to restart the kernel to use updated packages.


In [82]:
from safetensors.numpy import load_file
from pathlib import Path
from loguru import logger
import orjson
from rich import pretty
import plotly.graph_objects as go
from faker import Faker
from typing import Literal

# from IPython.display import display
import numpy as np

np.set_printoptions(
    precision=3,
    suppress=True,
    threshold=5,
)
rng = np.random.default_rng(0)
faker = Faker()

In [12]:
PROJ_ROOT = Path.cwd().resolve().parent
logger.info(PROJ_ROOT)
DATA_DIR = PROJ_ROOT / "data"
logger.info(DATA_DIR)
MLP_MSE_VALIDATION = DATA_DIR / "mlp_mse_validation"
MLP_MSE_VALIDATION_TENSORS = MLP_MSE_VALIDATION / "validation_mses.safetensors"
MLP_MSE_VALIDATION_META = MLP_MSE_VALIDATION / "validation_mses.json"

2024-12-13 14:18:35.910 | INFO     | __main__:<cell line: 0>:2 - /Users/gat/work/FA2024/embedding_translation
2024-12-13 14:18:35.911 | INFO     | __main__:<cell line: 0>:4 - /Users/gat/work/FA2024/embedding_translation/data


In [26]:
tensors = load_file(MLP_MSE_VALIDATION_TENSORS)
logger.info(list(tensors.keys()))
validation_mse = tensors["validation_mses"]
logger.info(tensors["validation_mses"].shape)

[12/13/24 14:33:27] INFO     [3396536193.py:2]                   ]8;id=747558;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_17291/3396536193.py\3396536193.py]8;;\:]8;id=363556;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_17291/3396536193.py#2\2]8;;\
                             ['validation_mses']                                
                    INFO     [3396536193.py:4] (6, 17, 17, 6)    ]8;id=420402;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_17291/3396536193.py\3396536193.py]8;;\:]8;id=307068;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_17291/3396536193.py#4\4]8;;\


In [28]:
with MLP_MSE_VALIDATION_META.open() as f:
    meta = orjson.loads(f.read())

pretty.pprint(meta)
# 6 Datasets (ArguAna, FiQA, SciDocs, NFCorpus, HotPotQA, Trec-COVID)
# 17 Native Embedding Spaces
# 17 Stitched Embedding Spaces
# 6 Architectures (2, 3, 4, 5, 6, or 7 layers)

{
│   'dataset2idx': {'arguana': 0, 'fiqa': 1, 'scidocs': 2, 'nfcorpus': 3, 'hotpotqa': 4, 'trec-covid': 5},
│   'model2idx': {
│   │   'WhereIsAI_UAE-Large-V1': 0,
│   │   'BAAI_bge-base-en-v1.5': 1,
│   │   'BAAI_bge-large-en-v1.5': 2,
│   │   'BAAI_bge-small-en-v1.5': 3,
│   │   'intfloat_e5-base-v2': 4,
│   │   'intfloat_e5-large-v2': 5,
│   │   'intfloat_e5-small-v2': 6,
│   │   'thenlper_gte-base': 7,
│   │   'thenlper_gte-large': 8,
│   │   'thenlper_gte-small': 9,
│   │   'sentence-transformers_gtr-t5-base': 10,
│   │   'sentence-transformers_gtr-t5-large': 11,
│   │   'mixedbread-ai_mxbai-embed-large-v1': 12,
│   │   'sentence-transformers_sentence-t5-base': 13,
│   │   'sentence-transformers_sentence-t5-large': 14,
│   │   'openai_text-embedding-3-large': 15,
│   │   'openai_text-embedding-3-small': 16
│   },
│   'num_layers': [2, 3, 4, 5, 6, 7]
}

In [56]:
# Plot heatmap for one architecture
dataset_i = 0
architecture_i = 0

example_validation_mse = validation_mse[dataset_i, :, :, architecture_i]
logger.info(example_validation_mse.shape)
pretty.pprint(example_validation_mse)

[12/13/24 14:53:57] INFO     [2716447018.py:8] (17, 17)          ]8;id=34588;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_17291/2716447018.py\2716447018.py]8;;\:]8;id=443200;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_17291/2716447018.py#8\8]8;;\


array([[1.   , 0.002, 0.001, ..., 0.002, 0.001, 0.001],
│      [0.001, 1.   , 0.001, ..., 0.002, 0.001, 0.001],
│      [0.001, 0.002, 1.   , ..., 0.002, 0.001, 0.001],
│      ...,
│      [0.001, 0.002, 0.001, ..., 1.   , 0.001, 0.001],
│      [0.001, 0.001, 0.001, ..., 0.002, 1.   , 0.001],
│      [0.001, 0.002, 0.001, ..., 0.002, 0.001, 1.   ]], dtype=float32)

In [99]:
rng = np.random.default_rng(0)


def heatmap(
    matrix: np.ndarray | None = None,
    labels: list | None = None,
    title: str = "DEFAULT TITLE",
    show_values: bool = True,
    value_format: Literal[".2f", ".2e"] = ".2f",
    width: int = 800,
    height: int = 600,
    xaxis_title: str | None = None,
    yaxis_title: str | None = None,
    log: bool = False,
    nan_diagnal: bool = False,
):
    if matrix is None:
        faker = Faker()
        shape = (10, 10)
        matrix = rng.random(shape)
        # logger.info(matrix)
    else:
        shape = matrix.shape

    if labels is None:
        labels = [faker.name() for _ in range(shape[0])]
        # logger.info("Using example labels and matrix")
        logger.info(labels)
    col_labels = labels
    row_labels = labels
    logger.info("Creating heatmap visualization...")

    # Convert input to numpy array and handle None values
    matrix = np.array(matrix, dtype=object)
    none_mask = matrix is None
    matrix = matrix.astype(float)
    if log:
        matrix = np.log10(matrix)
    matrix[none_mask] = np.nan
    if nan_diagnal:
        matrix[np.identity(matrix.shape[0], dtype=bool)] = np.nan

    # Format cell values if needed
    text_vals = None
    if show_values:
        text_vals = np.vectorize(lambda x: "N/A" if np.isnan(x) else f"{x:{value_format}}")(matrix)

    fig = go.Figure(
        data=[
            go.Heatmap(
                z=matrix,
                x=col_labels,
                y=row_labels,
                colorscale="Viridis",
                text=text_vals,
                texttemplate="%{text}" if show_values else None,
                textfont={"color": "black"},
                reversescale=False,
                showscale=True,
            )
        ],
        layout={
            "title": title,
            "width": width,
            "height": height,
            "xaxis": {
                "title": xaxis_title,
                "nticks": len(col_labels) if col_labels else None,
                "fixedrange": True,  # Disable zoom/pan
            },
            "yaxis": {
                "title": yaxis_title,
                "nticks": len(row_labels) if row_labels else None,
                "fixedrange": True,  # Disable zoom/pan
            },
            "dragmode": False,  # Disable dragging/panning
            "modebar": {
                "remove": [
                    "zoomIn2d",
                    "zoomOut2d",
                    "autoScale2d",
                    "resetScale2d",
                    "hoverClosestCartesian",
                    "hoverCompareCartesian",
                    "toggleSpikelines",
                ]
            },
        },
    )
    return fig


heatmap()

[12/13/24 15:23:46] INFO     [1064263312.py:28] ['Gerald        ]8;id=904166;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_17291/1064263312.py\1064263312.py]8;;\:]8;id=943911;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_17291/1064263312.py#28\28]8;;\
                             Davila', 'Christopher Nunez',                      
                             'Richard Archer', 'Jessica                         
                             Baldwin', 'Andrea Williams',                       
                             'Elizabeth Pratt', 'Jeremy                         
                             Torres', 'Brenda Franklin', 'Nancy                 
                             James', 'Anthony Parker']                          
                    INFO     [1064263312.py:31] Creating        ]8;id=87908;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_17291/1064263312.py\1064263312.py]8;;\:]8;id=373851;file:///var/fol

In [108]:
labels = sorted(meta["model2idx"], key=lambda k: meta["model2idx"][k])
matrix = example_validation_mse
heatmap(
    matrix,
    log=True,
    title="Example Validation MSE (log)",
    labels=labels,
)

[12/13/24 15:27:46] INFO     [1064263312.py:31] Creating        ]8;id=550212;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_17291/1064263312.py\1064263312.py]8;;\:]8;id=396787;file:///var/folders/6f/f4k9qbkd5nqfywwdhvwdtl5m0000gn/T/ipykernel_17291/1064263312.py#31\31]8;;\
                             heatmap visualization...                           
